<center> <H1>Stage B Assessment - Fall '23 </H1> </center>

<center> <H4> by </H4> </center>

<center> <H3> Yusuf Okunlola </H3> </center>

Data: Appliances Energy Prediction Dataset - https://drive.google.com/file/d/1Eru_UHVc3WLHVveC9Q8K9QUxlzYeHt18/view?usp=share_link

https://archive.ics.uci.edu/ml/machine-learning-databases/00374 

</br> The dataset for the remainder of this quiz is the Appliances Energy Prediction data. The data set is at 10 min for about 4.5 months. The house temperature and humidity conditions were monitored with a ZigBee wireless sensor network. Each wireless node transmitted the temperature and humidity conditions around 3.3 min. Then, the wireless data was averaged for 10 minutes periods. The energy data was logged every 10 minutes with m-bus energy meters. Weather from the nearest airport weather station (Chievres Airport, Belgium) was downloaded from a public data set from Reliable Prognosis (rp5.ru), and merged together with the experimental data sets using the date and time column. Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters). The attribute information can be seen below.

Attribute Information:

Date, time year-month-day hour:minute:second

Appliances, energy use in Wh

lights, energy use of light fixtures in the house in Wh

T1, Temperature in kitchen area, in Celsius

RH_1, Humidity in kitchen area, in %

T2, Temperature in living room area, in Celsius

RH_2, Humidity in living room area, in %

T3, Temperature in laundry room area

RH_3, Humidity in laundry room area, in %

T4, Temperature in office room, in Celsius

RH_4, Humidity in office room, in %

T5, Temperature in bathroom, in Celsius

RH_5, Humidity in bathroom, in %

T6, Temperature outside the building (north side), in Celsius

RH_6, Humidity outside the building (north side), in %

T7, Temperature in ironing room , in Celsius

RH_7, Humidity in ironing room, in %

T8, Temperature in teenager room 2, in Celsius

RH_8, Humidity in teenager room 2, in %

T9, Temperature in parents room, in Celsius

RH_9, Humidity in parents room, in %

To, Temperature outside (from Chievres weather station), in Celsius

Pressure (from Chievres weather station), in mm Hg

RH_out, Humidity outside (from Chievres weather station), in %

Wind speed (from Chievres weather station), in m/s

Visibility (from Chievres weather station), in km

Tdewpoint (from Chievres weather station), °C

rv1, Random variable 1, nondimensional

rv2, Random variable 2, nondimensional

In [1]:
# import libraries
import pandas as pd
import numpy as np

#  import preprocessing libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import  mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
       
# import regression models
from sklearn.linear_model import LinearRegression, Ridge, Lasso

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# load dataset
df = pd.read_csv("dataset/energydata_complete.csv")
print("Data imported successfully")

Data imported successfully


In [3]:
# inspect dataframe by printing out the first 5 rows
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [4]:
# check the shape of the dataset i.e. the number of rows and columns
df.shape

(19735, 29)

In [5]:
# information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [6]:
#  check for the percentage of missingness in each column
a = pd.DataFrame(df.isnull().sum() / len(df) *100, columns=['% of missing values'])
a

,% of missing values
date,0.0
Appliances,0.0
lights,0.0
T1,0.0
RH_1,0.0
T2,0.0
RH_2,0.0
T3,0.0
RH_3,0.0
T4,0.0


**Evaluation**

The dataset has 29 columns and 19735 rows. 

The dataset has 25 numerical columns and 4 categorical columns. 

The dataset has 0 missing values.

In [7]:
# descriptive statistics of the dataset
df.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,...,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,...,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,...,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,...,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,...,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,...,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,...,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,...,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


# Tasks

From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). 

What is the RMSE value in three d.p.?

In [8]:
# instantiate response and target variables
X = df[["T2"]]
y = df["T6"]

# Split data into feature matrix `X_train` and target vector `y_train`.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# normalize data using MinMaxScaler
ss = MinMaxScaler()
scalled_x_train = ss.fit_transform(X_train)
scalled_x_test = ss.transform(X_test)

# fit model to scaled data
model = LinearRegression().fit(scalled_x_train, y_train)

# predict on scaled test data
y_pred = model.predict(scalled_x_test)

# calculate RMSE score
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Square Error:', round(rmse, 3)) 

Root Mean Square Error: 3.633


Remove the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a  random state of 42 (for reproducibility). Normalize the dataset using the MinMaxScaler (Hint: Use the MinMaxScaler fit_transform and transform methods on the train and test set respectively). Run a multiple linear regression using the training set. 

Answer the following questions:

What is the Mean Absolute Error (in three decimal places) for the  training set?

### **Preparing the data**

In [9]:
# drop date and lights columns
df.drop(columns=["date", "lights"], inplace=True)

In [10]:
# The next step is to divide the data into "attributes" (inputs) and "labels" (outputs)
target = "Appliances"
X = df.drop(columns=[target])
y = df[target]

In [11]:
# split data into training and test sets with a 70:30 ratio and a random state of 42 (for reproducibility)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
# instantiate a minmax scaler for normalisation
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### **Training the Algorithm**
We have split our data into training and testing sets, and now is finally the time to train our algorithm. 

In [13]:
# instantiate LinearRegression model
linear_model = LinearRegression()

#fit the model to the training dataset
linear_model.fit(x_train, y_train)

LinearRegression()

In [14]:
# Predict on the training set
y_train_pred = linear_model.predict(x_train)

# Calculate Mean Absolute Error for the training set
mae_train = mean_absolute_error(y_train, y_train_pred)
print(f"Mean Absolute Error for the training set: {mae_train:.3f}")

Mean Absolute Error for the training set: 53.742


In [15]:
# Calculate Mean Squared Error for the training set
mse_train = mean_squared_error(y_train, y_train_pred)

# Calculate Root Mean Squared Error for the training set
rmse_train = np.sqrt(mse_train)
print(f"Root Mean Squared Error for the training set: {rmse_train:.3f}")

Root Mean Squared Error for the training set: 95.216


In [16]:
# Predict on the test set
y_test_pred = linear_model.predict(x_test)

# Calculate Mean Absolute Error for the test set
mae_test = mean_absolute_error(y_test, y_test_pred)
print(f"Mean Absolute Error for the test set: {mae_test:.3f}")


Mean Absolute Error for the test set: 53.643


In [17]:
# Calculate Mean Squared Error for the test set
mse_test = mean_squared_error(y_test, y_test_pred)

# Calculate Root Mean Squared Error for the test set
rmse_test = np.sqrt(mse_test)
print(f"Root Mean Squared Error for the test set: {rmse_test:.3f}")


Root Mean Squared Error for the test set: 93.640


Did the Model above overfit to the training set? 

Training Set:
- Mean Absolute Error (MAE): 53.742
- Root Mean Squared Error (RMSE): 95.216
  

Test Set:
- Mean Absolute Error (MAE): 53.643
- Root Mean Squared Error (RMSE): 93.640


The RMSE (Root Mean Squared Error) for how well our model predicts was a bit higher for the training data (95.216) compared to the test data (93.640). </br>Usually, if the model learns too much from the training data, it does worse on new data. </br> But in this case, the difference in RMSE is small, suggesting that the model has generalized well to unseen data. 

Therefore, the answer is No, the model did not overfit to the training set.

In [18]:
# Train a Ridge Regression model
ridge_model = Ridge()
ridge_model.fit(x_train, y_train)

# Predict on the test set
y_test_pred_ridge = ridge_model.predict(x_test)

# Calculate Root Mean Squared Error for the test set with Ridge Regression
rmse_test_ridge = np.sqrt(mean_squared_error(y_test, y_test_pred_ridge))
print(f"Root Mean Squared Error for the test set with Ridge Regression: {rmse_test_ridge:.3f}")

Root Mean Squared Error for the test set with Ridge Regression: 93.641


Is there any change to the root mean squared error (RMSE) when evaluated on the test set?



Yes, there is a change in the Root Mean Squared Error (RMSE) when evaluated on the test set.

- RMSE for the test set with Linear Regression model: 93.640
- RMSE for the test set with Ridge Regression: 93.709

The RMSE for the Ridge Regression model (93.709) is slightly higher than the RMSE for the Linear Regression model (93.640), indicating a slight increase in the RMSE when using Ridge Regression.

In [19]:
# Train a Lasso Regression model
lasso_model = Lasso()
lasso_model.fit(x_train, y_train)

# Obtain feature weights
feature_weights = lasso_model.coef_

# Count the number of features with non-zero weights
non_zero_features = np.sum(feature_weights != 0)
print(f"Number of features with non-zero weights: {non_zero_features}")


Number of features with non-zero weights: 21


In [20]:
# Predict on the test set with Lasso Regression
y_test_pred_lasso = lasso_model.predict(x_test)

# Calculate Root Mean Squared Error for the test set with Lasso Regression
rmse_test_lasso = np.sqrt(mean_squared_error(y_test, y_test_pred_lasso))
print(f"Root Mean Squared Error for the test set with Lasso Regression: {rmse_test_lasso:.3f}")


Root Mean Squared Error for the test set with Lasso Regression: 93.892
